In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report,ConfusionMatrixDisplay
from sklearn.inspection import permutation_importance
sns.set_theme()
warnings.filterwarnings("ignore")

#### Question 1 : Load the dataset “Titanic.xlsx” using the pandas function read_excel. Then explore the dataset and the variables. Try to understand the datatypes of each variable, number of null values and range of each variable. 

In [ ]:
data = pd.read_excel("titanic3.xls")
data.head()

#### Question 2 : For this task use the columns pclass,sex,age,sibsp,parch,fare as your features and survived as your target. 

#### Question 3 :	Now find the number of null values in the dataset and perform null value imputation. Perform 5 types of null imputation. (after completing this step, you should have 5 different versions of your data each using a different strategy to replace/remove missing values)
* Replace the missing values in the fare and age columns with their global means respectively
* Replace the missing values in the fare and age columns with an invalid value like -1
* Replace the missing values in the fare and age columns with random values sampled from the fare and age columns respectively. (So, if the age column has n missing values, randomly sample n not null values from the age column and then replace the missing values with the sampled values)
* Use the pclass column to decide on the missing values in the fare column and the sex column to decide on the missing values of the age columns
* Just drop all the missing values


#### Question 4 : Once the null values have been replaced, we can visualize the data. Try to make some plots or graphics using pandas or matplotlib or seaborn to identify  

* The relationship between age, survived and pclass 
* The relationship between fare, survived and pclass 
* The relationship between sex, survived and pclass 

#### Question 5 : Once you have these plot and graphics you can now begin with the modelling of the data. Divide the dataset into a suitable train-test split and then fit a SVM classifier model  (SVM) from sklearn. For the first experiment use the parameter kernel = “linear”.  

### Question 6 : Once the model is trained then find the following metrics. 

* Model score  
* Confusion matrix 
* Classification report using the sklearn library. This will give you various metrics 

#### Question 7 : Repeat the above step for all your datasets (remember the 5 versions of your dataset you created in step 3). Try to observe the difference in results if any (if you do not notice any changes then try to find out the importance of that feature using the function provided [step 10 - in the document or PDF and Question 9 in the python notebook]. It may happen that the feature is not important to the model so different null value replacement methods have no effect on the performance).

#### Question 8 Repeat your experiments for RBF kernel and observe the differences between accuracy and recall.

#### Question 9	Finally, for your SVM models an utility function is provided which plots the relative importances of the features provide to the model. Use that function to understand which features are more important to the model and how does this change for the SVM model with linear and RBF kernel

In [ ]:
# Utility function for plotting feature importances (use this only for SVM with Linear Kernel)
def f_importances(classifier, names):

    coef = abs(classifier.coef_[0])
    imp = coef
    imp,names = zip(*sorted(zip(imp,names)))
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    plt.show()

# Utility function for plotting feature importances (use this only for SVM with RBF Kernel)
def f_importances_RBF(classifier,names,X_test,y_test):
    features = np.array(names)
    perm_importance = permutation_importance(classifier, X_test, y_test)
    sorted_idx = perm_importance.importances_mean.argsort()
    plt.barh(features[sorted_idx], perm_importance.importances_mean[sorted_idx])
    plt.xlabel("Permutation Importance")